# Set up du jupyter notebook

In [ ]:
# Paramètre(s) du notebook

# VERBOSE=True
VERBOSE=True

OPTIONS=""
if not VERBOSE:
    OPTIONS="--quiet"

## Dépendances

In [ ]:
#library non présente directement dans le SSPCloud

!pip install geopandas $OPTIONS
!pip install plotly $OPTIONS
!pip install matplotlib

In [ ]:
# library externes

import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd 
import plotly.express as px
from IPython.core.display import display, HTML

#library internes

import enquete_sport as *
import equip_sport as *
import filosofi as *

# Données filosofi

## Scrapping

## Cleaning

## Visualisation

# Données de l'enquête des pratiques sportives

## Scrapping

## Cleaning

## Visualisation